In [9]:
from ultralytics import YOLO

# 사전 학습된 모델 로드
model = YOLO('yolov8n.pt')

# 이미지에 대해 객체 감지
results = model('C:/Users/xogus/Downloads/bus.jpg', device='cpu')

# 결과 시각화
results[0].show()



image 1/1 C:\Users\xogus\Downloads\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 50.4ms
Speed: 4.9ms preprocess, 50.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)


In [10]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

text = clip.tokenize(["a dog", "a candle", "a car"]).to(device)
image = torch.randn(1, 3, 224, 224).to(device)  # 테스트용 더미 이미지

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    similarity = (image_features @ text_features.T).softmax(dim=-1)

print("Similarity:", similarity)


Similarity: tensor([[0.6523, 0.0249, 0.3228]], device='cuda:0', dtype=torch.float16)


In [11]:
from ultralytics import YOLO
import cv2

# YOLOv8 모델 로드
model = YOLO("yolov8n.pt")

# 웹캠 열기 (0번 카메라)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 인식 수행 (CPU 사용)
    results = model(frame, device='cpu')

    # 결과 이미지 생성
    annotated_frame = results[0].plot()

    # 가장 높은 확률의 객체명 추출
    boxes = results[0].boxes
    if boxes and len(boxes.cls) > 0:
        class_id = int(boxes.cls[0])
        class_name = model.names[class_id]
        print("Top-1 Object:", class_name)

    # 화면 출력
    cv2.imshow("YOLOv8 Detection", annotated_frame)

    # 'q' 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


다음 단계는 3단계: CLIP을 활용한 객체명 → 음악 태그 유사도 계산이야. 지금까지 실시간 객체 인식이 되니까, 이제 인식된 객체명을 CLIP 텍스트 인코더에 넣고, 사전에 정의한 음악 태그들과의 의미 유사도를 계산

3단계: CLIP 기반 의미 유사도 계산
목표
YOLO에서 인식된 객체명 → CLIP 텍스트 임베딩

음악 태그 리스트 → CLIP 텍스트 임베딩

두 임베딩 사이 코사인 유사도 계산

가장 유사한 Top-N 태그 출력

In [12]:
import clip
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 장치 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# CLIP 모델 로드
model, preprocess = clip.load("ViT-B/32", device=device)

# 예시 객체명
object_name = "candle"

# 예시 음악 태그 리스트
music_tags = ["romantic", "hip-hop", "sad", "freedom", "jazz", "lofi", "party", "dreamy"]

# 객체명 + 태그 리스트 토크나이징
texts = [f"a {object_name}"] + music_tags
text_tokens = clip.tokenize(texts).to(device)

# 텍스트 임베딩
with torch.no_grad():
    text_features = model.encode_text(text_tokens)

# 객체 벡터와 태그 벡터 분리
object_vec = text_features[0].unsqueeze(0)  # shape: (1, 512)
tag_vecs = text_features[1:]               # shape: (N, 512)

# 유사도 계산
similarities = cosine_similarity(object_vec.cpu(), tag_vecs.cpu()).flatten()

# Top-N 유사도 높은 태그 추출
top_n = 3
top_indices = similarities.argsort()[::-1][:top_n]
top_tags = [music_tags[i] for i in top_indices]

# 출력
print(f"Object: {object_name}")
print(f"Top-{top_n} tags: {top_tags}")


Object: candle
Top-3 tags: ['romantic', 'dreamy', 'freedom']


In [13]:
import cv2
from ultralytics import YOLO
import clip
import torch
from sklearn.metrics.pairwise import cosine_similarity

# CLIP 장치 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# YOLOv8 모델 로드
yolo_model = YOLO("yolov8n.pt")

# 음악 태그 리스트 (확장 가능)
music_tags = ["romantic", "hip-hop", "sad", "freedom", "jazz", "lofi", "party", "dreamy"]

# 태그 벡터 미리 계산
tag_tokens = clip.tokenize(music_tags).to(device)
with torch.no_grad():
    tag_vecs = clip_model.encode_text(tag_tokens)

# 웹캠 열기
cap = cv2.VideoCapture(0)  # 0번 카메라

print("Press 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지 (YOLOv8)
    results = yolo_model(frame, device='cpu')
    annotated_frame = results[0].plot()

    # 가장 확률 높은 객체명 가져오기
    boxes = results[0].boxes
    if boxes and len(boxes.cls) > 0:
        class_id = int(boxes.cls[0])
        class_name = yolo_model.names[class_id]
        print(f"Detected Object: {class_name}")

        # CLIP 텍스트 유사도 계산
        texts = [f"a {class_name}"] + music_tags
        text_tokens = clip.tokenize(texts).to(device)

        with torch.no_grad():
            text_features = clip_model.encode_text(text_tokens)

        object_vec = text_features[0].unsqueeze(0)
        similarities = cosine_similarity(object_vec.cpu(), tag_vecs.cpu()).flatten()

        top_n = 3
        top_indices = similarities.argsort()[::-1][:top_n]
        top_tags = [music_tags[i] for i in top_indices]

        print(f"Top-{top_n} Music Tags for '{class_name}': {top_tags}")
        cv2.putText(annotated_frame, f"Tags: {', '.join(top_tags)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # 화면에 결과 출력
    cv2.imshow("Real-Time Music Tag Recommender", annotated_frame)

    # 'q' 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.


In [ ]:
import cv2
from ultralytics import YOLO

cap = cv2.VideoCapture(0)  # 다른 장치면 1 or 2

if not cap.isOpened():
    print("❌ 카메라 열기 실패")
    exit()

model = YOLO('yolov8n.pt')

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 프레임 읽기 실패")
        break

    results = model(frame, device='cpu')
    annotated_frame = results[0].plot()
    cv2.imshow("YOLO Webcam", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


❌ 카메라 열기 실패
❌ 프레임 읽기 실패


: 

In [1]:
import cv2

for i in range(5):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"✅ 사용 가능: 카메라 {i}번")
        cap.release()
    else:
        print(f"❌ 사용 불가: 카메라 {i}번")


❌ 사용 불가: 카메라 0번
❌ 사용 불가: 카메라 1번
❌ 사용 불가: 카메라 2번
❌ 사용 불가: 카메라 3번
❌ 사용 불가: 카메라 4번


In [ ]:
# 100곡 중 원하는 JSON 포맷으로 재구성
formatted_json_output = [
    {
        "spotify_id": song.get("spotify_id", ""),
        "title": song["title"],
        "album_name": song.get("album_name", ""),
        "release_date": song.get("release_date", ""),
        "duration": song.get("duration", ""),
        "artist": song["artist"],
        "genres": song.get("genres", [])
    }
    for song in top_100_korean_songs
    if (song["title"], song["artist"]) in [(s["title"], s["artist"]) for s in diverse_top_100]
]

# 저장
output_path = "/mnt/data/top_100_korean_songs_diverse.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(formatted_json_output, f, ensure_ascii=False, indent=2)

output_path
